# Self-Healing Data Pipelines: Automatically Handle and Reprocess Bad Data

This notebook is reference in:
- https://krijnvanrooijen.nl/blog/self-healing-data-pipeline-handling-bad-data/
- https://krijnvanderburg.medium.com/self-healing-data-pipeline-handling-bad-data-b95c28c5458b

Encountering bad data is inevitable in data pipelines. Schema mismatches, missing values, and inconsistent formatting are common causes of failures. The question isn't *if* bad data will occur, but *how frequently* it happens. However, rather than halting the entire pipeline when encountering bad data, the goal is to handle it gracefully so that the pipeline can process all valid incoming data without disruption.

This notebook demonstrates how to build resilient data pipelines that effectively manage bad data using PySpark. By implementing these strategies, pipelines can continue to run uninterrupted, minimizing downtime and issues.

## Key Strategies

1. **Isolate and store bad records** - Separate invalid data to process valid records without disruption
2. **Automate reprocessing** - Attempt to reprocess previously failed records in subsequent pipeline runs
3. **Maintain testing history** - Keep records of problematic data for future debugging and pipeline validation

## 1. Schema Definition and Bad Data Handling Setup

First, we define our schema with a special `_corrupt_record` field that captures malformed rows. We'll use PySpark's "PERMISSIVE" mode, which allows corrupt records to be captured rather than causing the entire pipeline to fail.

The PERMISSIVE mode is crucial for self-healing pipelines as it allows records that don't match the schema to be isolated in the `_corrupt_record` column instead of failing the entire job.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col

spark = SparkSession.Builder().appName("Self-Healing Pipeline").getOrCreate()

# Define schema for validation 
# The _corrupt_record field is a special column that captures malformed rows when using PERMISSIVE mode
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("_corrupt_record", StringType(), True)
])

25/04/20 08:37:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## 2. Data Preparation

Next, we'll create test data from two sources to simulate a real-world pipeline scenario:

1. **New incoming data** - Contains both valid records and records with schema violations
2. **Previously invalid records** - Represents data that failed in previous pipeline runs and is being reprocessed

This approach demonstrates how our pipeline can handle both new bad data and attempt to reprocess historical bad data in a single run.

In [2]:
# Create RDDs for test data
# In production, you would read from files instead of using test data
new_data_rdd = spark.sparkContext.parallelize([
    "1,John Doe,30",
    "2,Jane Smith,25",
    "3,Bob Johnson,35",
    "4,Alice Johnson,twenty"  # Invalid: age is not an integer
])

# Previously invalid records to reprocess
previous_invalid_rdd = spark.sparkContext.parallelize([
    "5,45,Charlie Brown",  # Invalid: id and age are position swapped
])

## 3. Data Loading with Schema Validation

Now, we load the data applying our predefined schema. Any records that don't match the expected structure will be automatically flagged. The schema validation process:

- Parses each row according to the defined schema
- Places properly formatted data in the appropriate columns
- Captures malformed rows in the `_corrupt_record` column for later handling
- Allows the pipeline to continue processing valid records without interruption

In [3]:
# Read data with schema validation
new_df = spark.read.schema(schema).csv(new_data_rdd)
previous_invalid_df = spark.read.schema(schema).csv(previous_invalid_rdd)

# Combine datasets for unified processing
combined_df = new_df.union(previous_invalid_df)

# Show all data including corrupt records
print("All records:")
combined_df.show()

All records:


+---+-------------+----+--------------------+
| id|         name| age|     _corrupt_record|
+---+-------------+----+--------------------+
|  1|     John Doe|  30|                NULL|
|  2|   Jane Smith|  25|                NULL|
|  3|  Bob Johnson|  35|                NULL|
|  4|Alice Johnson|NULL|4,Alice Johnson,t...|
|  5|           45|NULL|  5,45,Charlie Brown|
+---+-------------+----+--------------------+



## 4. Separating Valid and Invalid Records

With our data loaded, we can now separate valid records from invalid ones using the `_corrupt_record` column. This is the key step in isolating bad data for separate handling:

- Records with NULL in the `_corrupt_record` column are valid and can be processed normally
- Records with values in the `_corrupt_record` column contain the original malformed data and need special handling

This separation allows the pipeline to process valid records while preserving problematic data for future reprocessing.

In [4]:
# Separate valid from invalid records
valid_df = combined_df.filter(col("_corrupt_record").isNull()).drop("_corrupt_record")
invalid_df = combined_df.filter(col("_corrupt_record").isNotNull()).select("_corrupt_record")

## 5. Results Analysis

Let's examine the results of our data separation to confirm that:
1. Valid records have been properly identified and are ready for further processing
2. Invalid records have been isolated along with their original format for future debugging

In [5]:
# Show results
print("Valid records:")
valid_df.show()

print("Invalid records (original form):")
invalid_df.show()

Valid records:
+---+-----------+---+
| id|       name|age|
+---+-----------+---+
|  1|   John Doe| 30|
|  2| Jane Smith| 25|
|  3|Bob Johnson| 35|
+---+-----------+---+

Invalid records (original form):
+--------------------+
|     _corrupt_record|
+--------------------+
|4,Alice Johnson,t...|
|  5,45,Charlie Brown|
+--------------------+



## 6. Writing Results to Storage

The final step in our self-healing pipeline is to write both valid and invalid records to appropriate storage locations:

1. **Valid Records**: Written to the primary data destination in a structured format with headers
2. **Invalid Records**: Written to a separate location in their original form for later reprocessing

This approach ensures data processing continuity while maintaining a complete record of all incoming data.

In [6]:
# Write valid records as CSV with headers
valid_df.write.option("header", "true").mode("overwrite").csv("handle_bad_data/output/good")

# Write invalid records in their original form as text
# Using text format because _corrupt_record contains the original CSV line as string
invalid_df.write.option("header", "false").mode("overwrite").text("handle_bad_data/output/bad")

# Optionally, consider writing the bad data to a bad data history archive in append mode
#   to maintain a permanent track of bad data for testing the pipeline.
#   Not included in this archive code is deduplication; 
#   the same bad record might fail to be reprocessed multiple times.
# invalid_df.write.option("header", "false").mode("append").text("handle_bad_data/output/bad_history")

## Conclusion and Key Takeaways

This notebook has demonstrated how to build a self-healing data pipeline that gracefully handles bad data using PySpark. By implementing these techniques, data engineers can create more resilient systems that continue functioning even when encountering problematic data.

### Key Takeaways

1. **Isolate and Manage Bad Data**: Keep bad records separate from valid data to prevent pipeline failures.

2. **Automate Bad Data Reprocessing**: Automatically reprocess bad data when underlying issues are fixed, removing the need for manual intervention.

3. **Track Bad Data History**: Retain historical records of bad data to ensure that all data can be tested and properly processed in future pipeline versions.

By following these practices, data pipelines become more robust, capable of handling errors efficiently, and can evolve without being derailed by data quality issues.